<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [4]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT into houses values (?,?,?,?,?)', last_sale)
# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [5]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
c.executemany('INSERT into houses values (?,?,?,?,?)', recent_sales)

conn.commit()

Select all rows from houses

In [6]:
# A
h = c.execute('SELECT * \
FROM houses')
h.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [7]:
pwd

'C:\\Users\\Alice Chang\\OneDrive\\ASSETS-ALICE\\Mischellanous\\Data Science & AI\\IOD_MODULE 2_SQL'

In [8]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing_data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [9]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [10]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [11]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [12]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [13]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

In [14]:
#Answer: DELETE FROM houses would remove the rows of values from the table.

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [15]:
c.execute('SELECT * FROM houses WHERE bdrms = 4')
conn.commit()

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [16]:
# ANSWER
results = c.execute('SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms')
results.fetchall() # fetchall is a python command not an SQL function.

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [17]:
import pandas as pd
from pandas.io import sql

In [18]:
# Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [19]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing_data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [20]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [21]:
#ANSWER 

A = c.execute('SELECT *, AVG(price) AS Avg_price FROM houses_pandas GROUP BY bdrms ORDER BY Avg_price')
A.fetchall()

[(1000, 1, 5, 169900, 169900.0),
 (1416, 2, 49, 232000, 280866.6666666667),
 (2104, 3, 70, 399900, 326403.92),
 (3000, 4, 75, 539900, 377449.78571428574),
 (4478, 5, 49, 699900, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

In [22]:
# Answers to the discussion questions:
# 1. Use Pandas to check the columns and rows before storing the dataset from Pandas into a database.
# 2. Use Pandas to add or delete rows or columns in the dataset.  

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [23]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [24]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [25]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [26]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [27]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [28]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes 

In [29]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [30]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [31]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')
db = db_connection.cursor()

# Save these two dataframes as a table in sqlite
orders.to_sql('orders_sql', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down_sql', con = db_connection, if_exists = 'replace', index = False)
db_connection.commit()


#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [32]:
# Select first 10 rows of all columns in orders table

query = 'SELECT * FROM orders_sql LIMIT 10'
        
orders_10 = sql.read_sql(query, con=db_connection)
orders_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [33]:
# Select Number of Orders for Each Customer

query1 = 'SELECT customer_name, COUNT(order_id) FROM orders_sql GROUP BY customer_name'
        
customer_orders = sql.read_sql(query1, db_connection)
customer_orders


,customer_name,COUNT(order_id)
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8
...,...,...
787,Xylona Preis,2
788,Yana Sorensen,5
789,Yoseph Carroll,11
790,Zuschuss Carroll,9


#### 6. Select Number of Customers for Each Country

In [34]:
# Select Number of Customers for Each Country

query2 = 'SELECT country, COUNT(order_id) AS No_of_customers FROM orders_sql GROUP BY country'
        
customer_country = sql.read_sql(query2, db_connection)
customer_country



,country,No_of_customers
0,Austria,135
1,Belgium,68
2,Denmark,29
3,Finland,34
4,France,991
5,Germany,806
6,Ireland,50
7,Italy,493
8,Netherlands,194
9,Norway,37


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [35]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

query3 = 'SELECT discount, sales, quantity FROM orders_break_down_sql'
        
Orders_break_down = sql.read_sql(query3, db_connection)
Orders_break_down

,discount,sales,quantity
0,0.5,45.0,3
1,0.0,854.0,7
2,0.0,140.0,3
3,0.5,27.0,2
4,0.5,17.0,2
...,...,...,...
8042,0.0,245.0,2
8043,0.5,30.0,2
8044,0.5,23.0,4
8045,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [36]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table

query4 = 'SELECT discount, sales, quantity, (sales*quantity)-discount AS Total_Price FROM orders_break_down_sql'
        
Total_price = sql.read_sql(query4, db_connection)
Total_price

,discount,sales,quantity,Total_Price
0,0.5,45.0,3,134.5
1,0.0,854.0,7,5978.0
2,0.0,140.0,3,420.0
3,0.5,27.0,2,53.5
4,0.5,17.0,2,33.5
...,...,...,...,...
8042,0.0,245.0,2,490.0
8043,0.5,30.0,2,59.5
8044,0.5,23.0,4,91.5
8045,0.5,108.0,3,323.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [37]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query5 = 'SELECT * ,(sales*quantity)-discount AS Total_Price FROM orders_break_down_sql WHERE Total_Price > 100'
        
More_than_100 = sql.read_sql(query5, db_connection)
More_than_100

,order_id,product_name,discount,sales,quantity,category,Total_Price
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies,134.5
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture,5978.0
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies,420.0
3,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies,270.0
4,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies,828.0
...,...,...,...,...,...,...,...
6461,ES-2014-2815584,"Kleencut Shears, Serrated",0.0,261.0,6,Office Supplies,1566.0
6462,ES-2014-4785777,"Wilson Jones Index Tab, Economy",0.0,32.0,5,Office Supplies,160.0
6463,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture,490.0
6464,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology,323.5


#### 8. Select All Customer And The Product They Have Bought

In [38]:
# Select All Customer And The Product They Have Bought

query6 = 'SELECT o.customer_name, obd.product_name FROM orders_sql AS o INNER JOIN orders_break_down_sql AS obd \
ON o.order_id = obd.order_id'
        
Customers_products = sql.read_sql(query6, db_connection)
Customers_products

,customer_name,product_name
0,Eugene Moren,"Enermax Note Cards, Premium"
1,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,Grant Thornton,"Boston Markers, Easy-Erase"
4,Grant Thornton,"Eldon Folders, Single Width"
...,...,...
8042,Susan Vittorini,"Fellowes Shelving, Single Width"
8043,Dennis Pardue,"Wilson Jones Index Tab, Economy"
8044,Jim Kriz,"Avery Binder Covers, Recycled"
8045,Jim Kriz,"BIC Pencil Sharpener, Fluorescent"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [39]:
# Select Number of 'Furniture' Orders For Each Country
query7 = 'SELECT country AS Country, COUNT(category) AS Furniture_orders \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id WHERE category = "Furniture" GROUP BY country'
        
furniture_order_by_country = sql.read_sql(query7, db_connection)
furniture_order_by_country

,Country,Furniture_orders
0,Austria,40
1,Belgium,20
2,Denmark,9
3,Finland,16
4,France,299
5,Germany,264
6,Ireland,12
7,Italy,136
8,Netherlands,76
9,Norway,6


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [40]:
# Select Number of 'Furniture' Orders For The Country Denmark

query8 = 'SELECT country, COUNT(category) AS Furniture_orders \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id \
WHERE country = "Denmark" AND category = "Furniture" '
        
furniture_order_by_Denmark = sql.read_sql(query8, db_connection)
furniture_order_by_Denmark

,country,Furniture_orders
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [41]:
# Select Total Sales With Discount and Without Discount for Each Country
query9 = 'SELECT sales AS Total_sales, (sales-discount) AS Discounted_sales, country \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id \
GROUP BY country'    
Total_sales = sql.read_sql(query9, db_connection)
Total_sales

,Total_sales,Discounted_sales,country
0,55.0,55.0,Austria
1,12.0,12.0,Belgium
2,17.0,16.5,Denmark
3,195.0,195.0,Finland
4,140.0,140.0,France
5,17.0,17.0,Germany
6,185.0,184.5,Ireland
7,22.0,22.0,Italy
8,9.0,8.5,Netherlands
9,13.0,13.0,Norway


#### 11.A Select Total Quantity, Total Sales for Each Country

In [42]:
# Select Total Quantity, Total Sales for Each Country

query10 = 'SELECT sum(quantity) AS Total_quantity, sum(sales) AS Total_sales, country \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id \
GROUP BY country'    
Total_quantity_sales_per_country = sql.read_sql(query10, db_connection)
Total_quantity_sales_per_country

,Total_quantity,Total_sales,country
0,973,79382.0,Austria
1,532,42320.0,Belgium
2,204,7763.0,Denmark
3,201,20702.0,Finland
4,7329,609683.0,France
5,6179,488681.0,Germany
6,392,15998.0,Ireland
7,3612,252742.0,Italy
8,1526,70313.0,Netherlands
9,261,20529.0,Norway


#### 11.B Select Top 3 Country Based on Sales

In [43]:
# Select Top 3 Country Based on Sales

query10 = 'SELECT MAX(sales) AS Top_sales, country \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id \
GROUP BY country \
ORDER BY sales DESC \
LIMIT 3'    
Top_3 = sql.read_sql(query10, db_connection)
Top_3

,Top_sales,country
0,6517.0,Spain
1,5785.0,United Kingdom
2,5729.0,France


#### 11.C Select Bottom 3 Country Based On Quantities

In [44]:
# Select Bottom 3 Country Based On Quantities

query11 = 'SELECT MIN(quantity) AS bottom_qty, country \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id \
GROUP BY country \
ORDER BY quantity DESC \
LIMIT 3'    
Bottom_3 = sql.read_sql(query11, db_connection)
Bottom_3

,bottom_qty,country
0,1,United Kingdom
1,1,Switzerland
2,1,Sweden


#### 12. Select Average Sales By Categroy For The Country 'France'

In [53]:
# Select Average Sales By Category For The Country 'France'

query12 = 'SELECT category, AVG(sales) AS Avg_sales, country \
FROM orders_break_down_sql JOIN orders_sql \
ON orders_break_down_sql.order_id = orders_sql.order_id \
WHERE country = "France" \
GROUP BY category \
ORDER BY Avg_sales'
Avg_sales_by_category_France = sql.read_sql(query12, db_connection)
Avg_sales_by_category_France

,category,Avg_sales,country
0,Office Supplies,167.217709,France
1,Furniture,582.314381,France
2,Technology,595.145078,France


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [59]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

query13 = 'SELECT country, category, SUM(sales) AS Total_sales, AVG(sales) AS Avg_sales \
FROM orders_sql INNER JOIN orders_break_down_sql \
ON orders_sql.order_id = orders_break_down_sql.order_id \
GROUP BY country, category \
ORDER BY Avg_sales DESC \
LIMIT 1'
Highest_avg_total_sales = sql.read_sql(query13, db_connection)
Highest_avg_total_sales

,country,category,Total_sales,Avg_sales
0,Switzerland,Technology,11737.0,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



